In [1]:
import matplotlib
matplotlib.use('Agg')
from matplotlib import pyplot as plt

import generative_adversarial as gan
import dwdii_transforms
import numpy as np
import random

Using Theano backend.


In [2]:
imagePath = "/root/facial_expressions/images"
dataPath = "/root/facial_expressions/data/legend.csv"

### Load Data
In this section we load the image data and scale to 34x34.

In [4]:
maxData = 2200
X_data, Y_data = dwdii_transforms.load_data(dataPath, imagePath, maxData = maxData, verboseFreq = 200, imgResize=(34,34))
print X_data.shape
print Y_data.shape

0.000000: Jean_Chretien_0048.jpg
0.090909: Yoriko_Kawaguchi_0003.jpg
0.181818: Bo_Ryan_0001.jpg
0.272727: Pablo_Khulental_0001.jpg
0.363636: Jennifer_Capriati_0035.jpg
0.454545: Hiroki_Gomi_0001.jpg
0.545455: Ali_Naimi_0006.jpg
0.636364: Mohammad_Khatami_0008.jpg
0.727273: Simon_Cowell_0001.jpg
0.818182: Lindsay_Davenport_0001.jpg
(2200, 34, 34)
(2200, 1)


In [5]:
X_train, X_test, Y_train, Y_test = dwdii_transforms.trainingTestSplit(X_data, Y_data, 0.8, True)
shp = X_train.shape[1:]
print "Shape: " + str(shp)

1760
X_train: (1760, 1, 34, 34)
X_test: (440, 1, 34, 34)
Y_train: (1760, 1)
Y_test: (440, 1)
Shape: (1, 34, 34)


### Create Generative and Discriminative Models

In [6]:
genModel = gan.buildGenerativeModel(verbose=False)

In [7]:
discrimModel = gan.buildDiscriminativeModel(shape=shp, verbose=False)

### Discriminator Training

In [8]:
ntrain = 600
trainidx = random.sample(range(0,X_train.shape[0]), ntrain)
XT = X_train[trainidx,:,:,:]

In [9]:
# Pre-train the discriminator network ...
noise_gen = np.random.uniform(0,1,size=[XT.shape[0],100])
generated_images = genModel.predict(noise_gen)
X = np.concatenate((XT, generated_images))
n = XT.shape[0]
y = np.zeros([2*n,2])
y[:n,1] = 1
y[n:,0] = 1

In [10]:
gan.makeTrainable(discrimModel,True)
discrimModel.fit(X,y, nb_epoch=1, batch_size=128)

Epoch 1/1
1200/1200 [==============================] - 156s - loss: 7.2786   


In [11]:
y_hat = discrimModel.predict(X)

In [12]:
# Measure accuracy of pre-trained discriminator network
y_hat_idx = np.argmax(y_hat,axis=1)
y_idx = np.argmax(y,axis=1)
diff = y_idx-y_hat_idx
n_tot = y.shape[0]
n_rig = (diff==0).sum()
acc = n_rig*100.0/n_tot
print "Accuracy: %0.02f pct (%d of %d) right"%(acc, n_rig, n_tot)

Accuracy: 50.00 pct (600 of 1200) right


### Stacked GAN and Training

In [13]:
GAN = gan.buildStackedGanModel(genModel, discrimModel)

In [14]:
gan.makeTrainable(discrimModel,False)

In [ ]:
# Train for 6000 epochs at original learning rates
gan.train_for_n(X_train, GAN, genModel, discrimModel, nb_epoch=1000, plt_frq=50,BATCH_SIZE=32)
GAN.save_weights("dwdii-GAN-02.hdf5", overwrite=True)